# Extracting Text from Video using Python
2 Steps:
* Getting Audio from Video
* Getting Text from Audio

As an example I used video from open sources.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

ROOT = "/content/drive/MyDrive/Audio_to_Text"
GDRIVE = "/content/drive/MyDrive"

Mounted at /content/drive


In [ ]:
"""Download video file"""
# # import webdriver
# from selenium import webdriver
# from selenium.webdriver.common.by import By

# # create webdriver object
# driver = webdriver.Chrome()

# # get geeksforgeeks.org
# driver.get("https://disk.yandex.ru/d/pu9oHs3y9dIBQA/Презентация/Степ%201.%20Какие%20задачи%20возникают%20в%20NLP.mp4")

# # get element
# element = driver.find_element(By.CSS_SELECTOR, 'button.Button2 Button2_view_clear-inverse Button2_size_s download-button slider__button slider__button_download')

# # click the element
# element.click()



In [ ]:
# import re
# import requests
# from bs4 import BeautifulSoup
# import os.path
# from urllib.request import urlopen


# headers = {'user-agent': None}

# video_url = "https://disk.yandex.ru/d/pu9oHs3y9dIBQA/Презентация/Степ%201.%20Какие%20задачи%20возникают%20в%20NLP.mp4"

# def download_video(video_url: str):
#     # episode = re.search(r"bleach/(\d)\.1080", video_url).group(1)
#     filename = "Step_1.mp4"

#     print(f"Скачиваю видео: {filename}")

#     try:
#         response = requests.get(video_url, headers=headers, stream=True)
#         print('headers:', response.headers)
#         response.raise_for_status()

#         with open(VIDEO, "wb") as file:
#             for chunk in response.iter_content(chunk_size=8192):
#                 file.write(chunk)

#         print(f"Видео сохранено в файл {VIDEO}")

#     except requests.RequestException as ex:
#         print(f"Не удалось скачать видео: {ex}")


# def main() -> None:
#   download_video(video_url)

In [ ]:
"""
Specifically look at field Content-Type, it should have some binary type like application/octet-stream or video/mp4.

But definitely not some text format like application/json or text/html, text format file is definitely not video file.
In case if it is text/html then try renaming file to test.html and open it in browser to see what's there, probably server responded with some error inside this HTML.

Also just visually compare in some viewer content of two files - downloaded by script and downloaded by some downloader (e.g. browser).
Maybe there is some obvious problem visible by eye.

Also file size should be quite big for video.
If it is like 50KB then possibly some bad data is inside there.
"""
# main()

Скачиваю видео: Step_1.mp4
headers: {'Access-Control-Allow-Origin': 'yastatic.net', 'Content-Length': '13274', 'Content-Type': 'text/html', 'Set-Cookie': '_yasc=j6aXVnW/NzbuD7lKW5bhbvWoA3obUYvjM1uek3RiWx2hdH99+GZ3a8pU0Ie29JXmkg+Q; domain=.yandex.ru; path=/; expires=Fri, 07 Apr 2034 07:22:31 GMT; secure', 'X-Yandex-Captcha': 'captcha', 'X-Yandex-EU-Request': '0'}
Видео сохранено в файл /content/drive/MyDrive/Audio_to_Text/Step_1.mp4


## Step 1: Extracting Audio from Video

In [2]:
%%capture
!pip install moviepy

In [3]:
import os
from glob import glob
from tqdm.notebook import tqdm

from moviepy.editor import VideoFileClip

In [4]:
#Define the input video file and output audio file
ROOT = '/content/drive/MyDrive/Audio_to_Text'
VIDEO = f'{ROOT}/Video'
AUDIO = f'{ROOT}/Audio'
TEXT = f'{ROOT}/Text'

***

In [5]:
# Rename video files:

i = 1

for folder in tqdm(glob(f"{VIDEO}/*")):
  folder = os.rename(folder, f"{VIDEO}/Step_{i}.mp4")

  i += 1

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
glob(f"{VIDEO}/*")

['/content/drive/MyDrive/Audio_to_Text/Video/Step_1.mp4']

In [7]:
for video in glob(f"{VIDEO}/*"):
  print(video)

/content/drive/MyDrive/Audio_to_Text/Video/Step_1.mp4


In [8]:
example = '/content/drive/MyDrive/Audio_to_Text/Video/Step_1.mp4'
example.split('/')[-1].split('.')[0]

'Step_1'

In [9]:
"""
Writing Python code to extract Audio
"""
for video in glob(f"{VIDEO}/*"):
  video_clip = VideoFileClip(video)

  # Extract the audio from the video clip
  audio_clip = video_clip.audio

  # Write the audio to a separate file
  audio_name = video.split('/')[-1].split('.')[0]
  audio_clip.write_audiofile(f"{AUDIO}/{audio_name}.mp3")

  #Close the video and audio clips
  audio_clip.close()
  video_clip.close()

  print(f"Audio {audio_name} extraction successful!")


print("Audio extraction done!")

MoviePy - Writing audio in /content/drive/MyDrive/Audio_to_Text/Audio/Step_1.mp3


MoviePy - Done.
Audio Step_1 extraction successful!
Audio extraction done!


***

## Step 2: Getting Text from Audio

In [10]:
%%capture
!pip install -q --upgrade transformers accelerate

In [11]:
# Setting up the transcription model
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device used: {device}")

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

Device used: cuda


In [13]:
"""
Whisper is a pre-trained model for automatic speech recognition (ASR) and speech translation.
Trained on 680k hours of labelled data,
Whisper models demonstrate a strong ability to generalise to many datasets and domains without the need for fine-tuning.

Whisper predicts the language of the source audio automatically.
If the source audio language is known a-priori, it can be passed as an argument to the pipeline
"""
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
    )

model.to(device)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [14]:
processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
# Configuring the transcription pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device
)

***

In [16]:
for audio in glob(f"{AUDIO}/*"):
  print(audio)

/content/drive/MyDrive/Audio_to_Text/Audio/Step_1.mp3


In [17]:
example = '/content/drive/MyDrive/Audio_to_Text/Audio/Step_1.mp3'

example.split('/')[-1].split('.')[0]

'Step_1'

In [18]:
"""
Get Text from Audio
"""

for audio in glob(f"{AUDIO}/*"):
  text = pipe(audio)

  # Write text file
  text_name = audio.split('/')[-1].split('.')[0]

  with open(f'{TEXT}/{text_name}.txt', 'w', newline='', encoding='utf-8') as f:
    for sentence in text["text"].split('.'):
      # write each item on a new line
      f.write("%s\n" % sentence)

  print(f"Text {text_name} extraction successful!")


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


Text Step_1 extraction successful!


In [19]:
example_file = open(f'{TEXT}/Step_1.txt', 'r', encoding='utf-8')

print(example_file.read())

example_file.close()

 Введение в обработку естественного языка В последние несколько лет произошел бум технологии обработки естественного языка и сейчас мы все чаще и чаще в нашей повседневной жизни встречаемся с приложениями, которые используют автоматическую обработку текста
 Например, это автоматические переводчики, которые переводят с одного языка на другой как текст, так и речь
 Это диалоговые ассистенты, которые сейчас применяются банками, крупными компаниями
 И даже мы в университете в прошлом году применяли диалоговый ассистент для приемной компании
 Приложению диалоговому ассистенту можно задать вопрос текстом или голосом, и он ответит также голосом или текстом еще один пример приложение обработки естественного языка это автоматическая генерация текста здесь вы можете видеть пример системы яндекса балабола которая продолжает фразу которую вы начали и генерирует текст но конечно же генерация текста используется и для серьезных приложений например в тех же самых диалоговых ассистентах
 Ну и здесь мы